In [15]:
import pandas as pd
import yfinance as yf
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.hierarchical_portfolio import HRPOpt
from datetime import date

In [2]:

'''

SP - 500 Consituents

'''


sp500 = pd.read_csv('sp500_holdings_june_2022.csv')[:505][['Ticker', 'Sector']]


In [38]:


folio = yf.download(["AAPL", "SG", "ACLS", "APO", "V", "AVAV"], '2010-01-04', progress = False)['Close']



In [4]:


mu = mean_historical_return(folio)
S = CovarianceShrinkage(folio).ledoit_wolf()



In [39]:

HRP = HRPOpt(folio.pct_change())

HRP.optimize()



OrderedDict([('AAPL', 0.2525203169075635),
             ('ACLS', 0.0591940548491706),
             ('APO', 0.15531120525230344),
             ('AVAV', 0.14329582375748265),
             ('SG', 0.014719290254988285),
             ('V', 0.37495930897849156)])

In [40]:


HRP.portfolio_performance(True)



Expected annual return: 20.8%
Annual volatility: 23.4%
Sharpe Ratio: 0.80


(0.20811208023344466, 0.23426219283356745, 0.8029980337761531)

In [22]:

ef = EfficientFrontier(mu, S)

ef.add_constraint(lambda x : x >= 0.05)
ef.add_constraint(lambda x : x <= 0.45)

weights = ef.max_sharpe()

ef.portfolio_performance(verbose=True)


Expected annual return: 35.0%
Annual volatility: 30.4%
Sharpe Ratio: 1.09


(0.3502987128552621, 0.3039236136205092, 1.086782000649959)

In [6]:

weights


OrderedDict([('AAPL', 0.45),
             ('ACLS', 0.0500000000000001),
             ('ACMR', 0.45),
             ('SG', 0.05)])

In [28]:


def get_cash_asset_ratio(tick):

    try:
        return tick.balance_sheet.T['Cash'][0] / tick.balance_sheet.T['Total Assets'][0]
    except:
        return None





In [31]:
aapl = yf.Ticker("AAPL")

#aapl.get_recommendations()

#aapl.option_chain()[0]

In [32]:


get_cash_asset_ratio(aapl)



0.09954359234420317

In [57]:

sector_groups = {sec : {'Cash_Asset_Ratio' : []} for sec in sp500.Sector.unique()}

for (t, sec) in zip(sp500.Ticker, sp500.Sector):
    
    print(t)
    sector_groups[sec]['Cash_Asset_Ratio'].append(get_cash_asset_ratio(yf.Ticker(t)))


AAPL
MSFT
AMZN
GOOGL
GOOG
TSLA
BRK.B
UNH
JNJ
NVDA
FB
XOM
JPM
PG
V
CVX
HD
PFE
MA
ABBV
BAC
KO
LLY
PEP
MRK
AVGO
VZ
TMO
COST
ABT
CMCSA
ADBE
DIS
CSCO
MCD
ACN
WMT
WFC
INTC
BMY
PM
LIN
AMD
TXN
DHR
CRM
T
QCOM
COP
NEE
NKE
AMGN
RTX
UNP
MDT
HON
LOW
UPS
CVS
SPGI
ANTM
IBM
MS
AMT
INTU
CAT
ORCL
GS
LMT
C
SCHW
AMAT
AXP
MO
DE
PYPL
BLK
ADP
BKNG
NOW
PLD
CB
CI
DUK
MDLZ
ADI
SBUX
NFLX
GE
MMM
GILD
CCI
ISRG
SO
MMC
SYK
MU
ZTS
EOG
TGT
TMUS
TJX
REGN
PNC
BDX
LRCX
CME
USB
BA
CSX
NOC
D
VRTX
PGR
CL
SLB
SHW
TFC
FIS
PXD
EW
ATVI
OXY
WM
EQIX
CHTR
FISV
AON
ITW
HUM
MPC
BSX
NSC
FCX
EL


In [56]:


sector_groups



{'Information Technology': {'Cash_Asset_Ratio': 0.2236543301771938},
 'Consumer Discretionary': {'Cash_Asset_Ratio': 0.3344661493498011},
 'Communication Services': {'Cash_Asset_Ratio': 0.13332538310178285},
 'Financials': {'Cash_Asset_Ratio': 0.05801943363438334},
 'Health Care': {'Cash_Asset_Ratio': None},
 'Energy': {'Cash_Asset_Ratio': 0.02270164624520785},
 'Consumer Staples': {'Cash_Asset_Ratio': 0.18613104005321424},
 'Materials': {'Cash_Asset_Ratio': 0.09005827300017659},
 'Utilities': {'Cash_Asset_Ratio': 0.000453070009474067},
 'Industrials': {'Cash_Asset_Ratio': 0.033689341265859075},
 'Real Estate': {'Cash_Asset_Ratio': 0.10194399727380997}}